# Arithmetic Operations on Images
산술 연산
## Goal

* Learn several arithmetic operations on images like 
    * addition, +
    * subtraction, -
    * bitwise operations 비트연산자
    * etc.
* You will learn these functions : 
    * `cv2.add()`, 
    * `cv2.addWeighted()` 
    * etc.

## Image Addition

You can add two images by **OpenCV** function, `cv2.add()` or simply by numpy operation, `res = img1 + img2`. Both images should be of same depth and type, or second image can just be a scalar value.

> ## Note
>
> There is a difference between `OpenCV addition` and `Numpy addition`. 
> `OpenCV addition` is a saturated operation while `Numpy addition` is a modulo operation.

For example, consider below sample:

In [1]:
import cv2
import numpy as np

In [2]:
x = np.uint8([250])#2^8만큼 표현 가능
y = np.uint8([10])

# 250+10 = 260 => 255
print(cv2.add(x,y))#더했을때 최대 255

# 250+10 = 260 % 256 = 4
print(x+y)#최대 256까지 표현이 가능하기 때문에 256으로 나눈 나머지 값이 나옴.

[[255]]
[4]


It will be more visible when you add two images. **OpenCV function** will provide a better result. So, always better stick to OpenCV functions.

## Image Blending
이미지 더하기.

This is also image addtion, but different weights are given to images so that it gives a feeling of blending or trasparency. Images are added as per the equation below:
더할때 가중치를 두어서 섞이거나 투명한 느낌을 줌.

$$
g(x) = (1-\alpha) f_0 (x) + \alpha f_1 (x)
$$

By varing $\alpha$ from 0 to 1, you can perform a cool transition b/w one image to another

Here I took two images to blend them together. First image is given a weight of 0.7 and second image is given a weight of 0.3. `cv2.addWeight()` applies following equation on the image.

$$
dst = \alpha \cdot img_1 + \beta \cdot img_2 + \gamma
$$

Here $\gamma$ is taken as zero

In [3]:
img1 = cv2.imread('ml.png')
img2 = cv2.imread('opencv-logo.png')
img2 = cv2.resize(img2, dsize=(img1.shape[1],img1.shape[0]), interpolation= cv2.INTER_AREA)
print(img1.shape)
print(img2.shape)

dst = cv2.addWeighted(img1,0.7, img2,0.3,0)
#img1 에 가중치 0.7, img2에 가중치 0.3
#img1이 더 진한 색으로 나타나게 됨.

cv2.imshow('dst',dst)
cv2.waitKey(0)
cv2.destroyWindow('dst')

(380, 308, 3)
(380, 308, 3)


## Bitwise Operations

This includes bitwise **AND**, **OR**, **NOT** and **XOR** operations. 
They will be highly useful while extracting any part of the image (as we will see in coming chapters), defining and working with non-rectangular ROI etc.
이미지의 일부를 추출할때, ROI를 정의할때. ROI(관심 영역, 영상 내에서 작업을 수행하고자 하는 특정 영역을 의미)
Below we will see an example on how to change a particular region of an image.

I want to put **OpenCV** logo above an image. If I add two images, it will change color. If I blend it, I get an transparent effect. But I want it to be opaque. It it was a rectangular region, I could use ROI as we did in last chapter. But OpenCV log is a not a retangular shape. So you can do it with bitwise operations as below:

두개 이미지를 그냥 더하면 색이 바뀌고, blend를 하면 투명한 효과를 얻을 수 있다.
불투명하게 합성을 하고 싶을 때, 직사각형 모양이 아닌 이미지를 비트연산자로 수행할 수 있는 것.

In [7]:
# Load two images
import numpy as np
import cv2

img1 = cv2.imread('messi5.jpg')
img2 = cv2.imread('opencv-logo-white.png')
img2 = cv2.resize(img2, dsize=(0,0), fx=0.2, fy=0.2, interpolation= cv2.INTER_AREA)

# I want to put logo on top-left corner, So I create a ROI
rows, cols, channels = img2.shape
roi = img1[0:rows, 0:cols]
#로고 위치 지정

# Now create a mask of logo and create its inverse mask also
img2gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
cv2.imshow('img2gray',img2gray)

ret, mask = cv2.threshold(img2gray, 10,255, cv2.THRESH_BINARY)
mask_inv = cv2.bitwise_not(mask)
cv2.imshow('mask',mask)

# Now black-out the area of logo in ROI
img1_bg = cv2.bitwise_and(roi,roi,mask = mask_inv)
#ROI에서 로고에 해당하는 부분만 검정색으로 만들기
#bitwise_and 연산자는 둘다 0이 아닌 경우만 값을 통과 시킴.
#즉 mask가 검정색이 아닌 경우만 통과가 되기때문에 mask영역 이외는 모두 제거됨.

# Take only region of logo from logo image
img2_fg = cv2.bitwise_and(img2,img2,mask= mask)
#로고 이미지에서 로고 부분만 추출하기

# Put logo in ROI and modify the main image
dst = cv2.add(img1_bg,img2_fg)
img1[0:rows,0:cols] = dst

cv2.imshow('res',img1)

cv2.waitKey(0)
cv2.destroyAllWindows()

For more understanding, display all the intermediate images in the above code, especially `img1_bg` and `img2_fg`.

